Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import os
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [19]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

### Convnet: Two Convolutional Layers, One Fully Connected Layer and Softmax
Let's build a small network with two convolutional layers, followed by one fully connected layer.
Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.


* **Architecture**

![4_convolutions_network1](images/4_convolutions_network1.png)



* **Weight Initialization**
    * Initialize weights with a small amount of noise
        * Symmetry breaking
        * Prevent 0 gradients
    * Make the noise positive to avoid "dead neurons" (we're using ReLU neurons)
        * Use stddev=0.1
        

* **Output Size**
    * $Out_{size}=\frac{In_{size} - W_{size} - 2P}{S} +1$
        * $Out_{size}$: output height and length
        * $In_{size}$: input height and length
        * $W_{size}$: patch size
        * $P$: padding
            * "same" = -1
            * "valid" = 0
        * $S$: stride



In [5]:
def compute_output_size(in_size, conv_patch_size, conv_stride, padding='same', pooling_patch_size=None, pooling_stride=None):
    if padding == "same":
        padding = -1
    elif padding == "valid":
        padding = 0
    else:
        raise ValueError('Unhandled padding {}'.format(padding))

    out = (((in_size - conv_patch_size - 2.0 * padding) / conv_stride) + 1)
    
    if pooling_patch_size is not None and pooling_stride is not None:
        out = (((out - pooling_patch_size - 2.0 * padding) / pooling_stride) + 1)
    
    return out
    
         

In [6]:
n_convs = 2
final_size = 28

for i in range(n_convs):
    final_size = compute_output_size(in_size=final_size, conv_patch_size=5, conv_stride=2, padding="same")

final_size = int(np.ceil(final_size))
print('Final _size after convolutions: ', final_size)

Final _size after convolutions:  7


Therefore the final hidden layer (ReLU layer) will have dimension [784,64]

$$(final\_size)(final\_size)(16) = (7)(7)(16) = 784$$

In [36]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    
    # Layer1 : 2D-Convolution
    # Input channels: num_channels = 1 (depth of the image)
    # Output channels: depth = 16
    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))

    # Layer2 : 2D-Convolution
    # Input channels: num_channels = 16 (depth of previous layer)
    # Output channels: depth = 16
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    # Layer3 : ReLU layer
    layer3_weights = tf.Variable(tf.truncated_normal(
      [final_size * final_size * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    # Layer4 : Softmax (output layer)
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Stride
    # 1-D tensor of length 4. 
    # The stride of the sliding window for each dimension of input.
    #     [batch, in_height, in_width, in_channels]
    stride = [1, 2, 2, 1]
    
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, filter=layer1_weights, strides=stride, padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        
        conv = tf.nn.conv2d(hidden, filter=layer2_weights, strides=stride, padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
    )

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [37]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 50 == 0):
          print('Minibatch loss at step %d: %f' % (step, l))
          print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
          print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.574154
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.967381
Minibatch accuracy: 37.5%
Validation accuracy: 39.4%
Minibatch loss at step 100: 1.321999
Minibatch accuracy: 37.5%
Validation accuracy: 68.1%
Minibatch loss at step 150: 0.728416
Minibatch accuracy: 81.2%
Validation accuracy: 73.4%
Minibatch loss at step 200: 0.942281
Minibatch accuracy: 62.5%
Validation accuracy: 75.3%
Minibatch loss at step 250: 2.161945
Minibatch accuracy: 37.5%
Validation accuracy: 72.4%
Minibatch loss at step 300: 0.570672
Minibatch accuracy: 81.2%
Validation accuracy: 77.0%
Minibatch loss at step 350: 0.578733
Minibatch accuracy: 87.5%
Validation accuracy: 79.0%
Minibatch loss at step 400: 1.060391
Minibatch accuracy: 68.8%
Validation accuracy: 79.9%
Minibatch loss at step 450: 0.736238
Minibatch accuracy: 81.2%
Validation accuracy: 80.1%
Minibatch loss at step 500: 0.737338
Minibatch accuracy: 81.2%
Validation accuracy: 79.2%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [58]:
n_convs = 2
final_size = image_size

for i in range(n_convs):
    final_size = compute_output_size(in_size=final_size, conv_patch_size=5, conv_stride=1, 
                                     pooling_patch_size=2, pooling_stride=2, padding="same")

final_size = int(np.ceil(final_size))
print('Final _size after convolutions: ', final_size)

Final _size after convolutions:  7


Therefore the final hidden layer (ReLU layer) will have dimension [784,64]

$$(final\_size)(final\_size)(16) = (7)(7)(16) = 784$$

In [ ]:
batch_size = 16
depth = 16
num_hidden = 64

# Convolutions
patch_size = 5
conv_stride = [1, 1, 1, 1]
padding='SAME'

# Pooling
pooling_stride = [1, 2, 2, 1]
pooling_patch = [1, 2, 2, 1]

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    
    # Layer1 : 2D-Convolution
    # Input channels: num_channels = 1 (depth of the image)
    # Output channels: depth = 16
    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))

    # Layer2 : 2D-Convolution
    # Input channels: num_channels = 16 (depth of previous layer)
    # Output channels: depth = 16
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    # Layer3 : ReLU layer
    layer3_weights = tf.Variable(tf.truncated_normal(
      [final_size * final_size * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    # Layer4 : Softmax (output layer)
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Stride
    # 1-D tensor of length 4. 
    # The stride of the sliding window for each dimension of input.
    #     [batch, in_height, in_width, in_channels]
    
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, filter=layer1_weights, strides=conv_stride, padding=padding)
        hidden = tf.nn.relu(conv + layer1_biases)
        pooling = tf.nn.max_pool(hidden, ksize=pooling_patch, strides=pooling_stride, padding=padding)
                
        conv = tf.nn.conv2d(pooling, filter=layer2_weights, strides=conv_stride, padding=padding)
        hidden = tf.nn.relu(conv + layer2_biases)
        pooling = tf.nn.max_pool(hidden, ksize=pooling_patch, strides=pooling_stride, padding=padding)
        
        shape = pooling.get_shape().as_list()
        reshape = tf.reshape(pooling, [shape[0], shape[1] * shape[2] * shape[3]])
        
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
    )

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [100]:
num_steps = 30000

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 5000 == 0):
          print('Minibatch loss at step %d: %f' % (step, l))
          print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
          print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.695924
Minibatch accuracy: 6.2%
Validation accuracy: 10.5%
Minibatch loss at step 5000: 0.700120
Minibatch accuracy: 75.0%
Validation accuracy: 87.9%
Minibatch loss at step 10000: 0.079954
Minibatch accuracy: 93.8%
Validation accuracy: 88.9%
Minibatch loss at step 15000: 0.281704
Minibatch accuracy: 87.5%
Validation accuracy: 89.3%
Minibatch loss at step 20000: 0.306930
Minibatch accuracy: 93.8%
Validation accuracy: 89.7%
Minibatch loss at step 25000: 0.408342
Minibatch accuracy: 87.5%
Validation accuracy: 89.9%
Test accuracy: 95.7%


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

### Attempt to create a better model:

**Architecture**
    $$Convolution > Max Pooling > Convolution > Max Pooling > Fully Connected > Fully Connected > Readout$$ 

**Configuration**
* Convolution Configuration
    * Patch size = 5
    * Padding: SAME
    * ReLU activations
* Max Polling Configuration
    * Stride = 2
    * Patch size = 2
    * Padding: VALID
* Fully Connected Layer Configuration
    * Neurons: [64, 32]
    * ReLU activations
    * Dropout: 50%
* Read-out
    * Softmax

**Loss**
* Softmax cross entropy with logits
* L2 regularization
    * Beta = 0.001

**Optimization**
* AdaDelta

* Learning rate
    * Initial value: 0.5
    * Exponential decay


In [7]:
n_convs = 2
final_size = image_size

for i in range(n_convs):
    final_size = compute_output_size(in_size=final_size, conv_patch_size=5, conv_stride=1, 
                                     pooling_patch_size=2, pooling_stride=2, padding="valid")

final_size = int(np.ceil(final_size))
print('Final _size after convolutions: ', final_size)

Final _size after convolutions:  4


Therefore the final hidden layer (ReLU layer) will have dimension [256,64]

$$(final\_size)(final\_size)(16) = (4)(4)(16) = 256$$

In [14]:
def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
            
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)


def weight_variable(size, distribution=tf.truncated_normal, **kwargs):
    return tf.get_variable('weight', initializer=distribution(size, **kwargs))


def bias_variable(size, distribution=tf.zeros, **kwargs):
    return tf.get_variable('biases', initializer=distribution(size, **kwargs))


def fully_connected_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu,
                          reuse_regularizers=None, reuse_weights=None):
    """Reusable code for making a simple neural net layer.

    It does a matrix multiply, bias add, and then uses relu to nonlinearize.
    It also sets up name scoping so that the resultant graph is easy to read,
    and adds a number of summary ops.
    """
    
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.variable_scope(layer_name, reuse=reuse_weights):
        # This Variable will hold the state of the weights for the layer

        weights = weight_variable([input_dim, output_dim],  stddev=0.1)
        variable_summaries(weights)
        
        biases = bias_variable([output_dim])
        variable_summaries(biases)

        preactivate = tf.matmul(input_tensor, weights) + biases
        tf.summary.histogram('pre_activations', preactivate)
            
        activations = act(preactivate, name='activation')
        tf.summary.histogram('activations', activations)

    with tf.variable_scope('regularization', reuse=reuse_regularizers):
        regularizers = tf.get_variable('regularizers', [], initializer=tf.zeros_initializer)
        regularizers += tf.nn.l2_loss(weights)
        
    return activations
    

def convolution_layer(input_tensor, num_channels, depth, conv_patch_size, conv_stride,padding, layer_name, 
                      act=tf.nn.relu, pool_patch_size=None, pool_stride=None, 
                      reuse_regularizers=None, reuse_weights=None):

    # Adding a name scope ensures logical grouping of the layers in the graphs    
    with tf.variable_scope(layer_name, reuse=reuse_weights):
        
        weights = weight_variable([conv_patch_size, conv_patch_size, num_channels, depth],  stddev=0.1)
        variable_summaries(weights)

        biases = bias_variable([depth])
        variable_summaries(biases)
    
        preactivate = tf.nn.conv2d(input_tensor, filter=weights, strides=conv_stride, padding=padding) + biases
        tf.summary.histogram('pre_activations', preactivate)

        activations = act(preactivate, name='activation')
        
        if pool_patch_size is not None and pool_stride is not None:
            activations = tf.nn.max_pool(activations, ksize=pool_patch_size, strides=pool_stride, 
                                         padding=padding, name='maxpool')
            
        tf.summary.histogram('activations', activations)
            
    with tf.variable_scope('regularization', reuse=reuse_regularizers):
        regularizers = tf.get_variable('regularizers', [], initializer=tf.zeros_initializer)
        regularizers += tf.nn.l2_loss(weights)
        
    return activations


def model(data, train=False):
    
    # Layer1 : 2D-Convolution
    # Input channels: num_channels = 1 (depth of the image)
    # Output channels: depth = 16
    conv = convolution_layer(
        layer_name="Conv1",
        input_tensor=data, num_channels=int(data.get_shape()[-1]), depth=depth, 
        conv_patch_size=conv_patch_size, conv_stride=[1,1,1,1], padding=padding,
        act=tf.nn.relu, pool_patch_size=pool_patch_size, pool_stride=pool_stride,
        reuse_regularizers=None if train else True, reuse_weights=None if train else True
    )


    # Layer2 : 2D-Convolution
    # Input channels: num_channels = 16 (depth of previous layer)
    # Output channels: depth = 16
    conv = convolution_layer(
        layer_name="Conv2",
        input_tensor=conv, num_channels=depth, depth=depth, 
        conv_patch_size=conv_patch_size, conv_stride=[1,1,1,1], padding=padding,
        act=tf.nn.relu, pool_patch_size=pool_patch_size, pool_stride=pool_stride,
        reuse_regularizers=True, reuse_weights=None if train else True
    )

    shape = conv.get_shape().as_list()
    reshape = tf.reshape(conv, [shape[0], shape[1] * shape[2] * shape[3]])

    # Layer3 : ReLU layer
    hidden = fully_connected_layer(
        layer_name="FullyConnected1",
        input_tensor=reshape, 
        input_dim=final_size * final_size * depth, output_dim=num_hidden,
        act=tf.nn.relu,
        reuse_regularizers=True, reuse_weights=None if train else True        
    )           

    if train:
        hidden = tf.nn.dropout(hidden, keep_prob=dropout_keep_prob)

    # Layer4 : ReLU layer
    hidden = fully_connected_layer(
        layer_name="FullyConnected2",
        input_tensor=hidden, 
        input_dim=num_hidden, output_dim=num_hidden/2,
        act=tf.nn.relu,
        reuse_regularizers=True, reuse_weights=None if train else True
    )

    if train:
        hidden = tf.nn.dropout(hidden, keep_prob=dropout_keep_prob)

    # Layer5 : Output layer
    logits = fully_connected_layer(
        layer_name="Read-out",
        input_tensor=hidden, 
        input_dim=num_hidden/2, output_dim=num_labels,
        act=tf.identity,
        reuse_regularizers=True, reuse_weights=None if train else True
    )

    return logits

In [18]:
batch_size = 16
depth = 16
num_hidden = 64

conv_patch_size = 5
conv_stride = [1, 1, 1, 1]

pool_patch_size = [1, 2, 2, 1]
pool_stride = [1, 2, 2, 1]

padding='VALID'
dropout_keep_prob = 0.5
beta = 0.001
start_learning_rate = 0.5

batch_size = 16


graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    # Validation and test data
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Training computation.
    # We compute the cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = model(tf_dataset, train=True)
    
    with tf.variable_scope('regularization', reuse=True):
        regularizers = tf.get_variable('regularizers', [], initializer=tf.zeros_initializer)

    with tf.name_scope('cross_entropy'):
        loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf_labels, logits=logits)
        loss = tf.reduce_mean(loss + beta * regularizers)
    
    tf.summary.scalar('cross_entropy', loss)

    # Addaptive learning rate. Source [https://www.tensorflow.org/api_docs/python/tf/train/exponential_decay]
    # Initialize counter variable for the number of steps taken
    with tf.name_scope('learning_rate'):
        global_step = tf.Variable(0, trainable=False)  
        learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 100000, 0.96, staircase=True)

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    # Passing global_step to minimize() will increment it at each step.
    with tf.name_scope('train'):
        optimizer = tf.train.AdadeltaOptimizer(learning_rate).minimize(loss, global_step=global_step)

    with tf.name_scope('prediction'):
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = tf.nn.softmax(model(tf_valid_dataset, train=False))
        test_prediction = tf.nn.softmax(model(tf_test_dataset, train=False))
        
    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            train_correct_prediction = tf.equal(tf.argmax(tf_labels, 1), tf.argmax(train_prediction, 1))
        with tf.name_scope('accuracy'):
            train_accuracy = tf.reduce_mean(tf.cast(train_correct_prediction, tf.float32)) * 100
        tf.summary.scalar('train_accuracy', train_accuracy)    

        with tf.name_scope('correct_prediction'):
            valid_correct_prediction = tf.equal(tf.argmax(valid_labels, 1), tf.argmax(valid_prediction, 1))
        with tf.name_scope('accuracy'):
            valid_accuracy = tf.reduce_mean(tf.cast(valid_correct_prediction, tf.float32)) * 100
        tf.summary.scalar('valid_accuracy', valid_accuracy)    
    
    # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
    merged_summary = tf.summary.merge_all()

In [20]:
num_steps = 40000

logs_path = os.environ['TENSORBOARD_LOGDIR']
writer = tf.summary.FileWriter(logs_path, graph=graph)

with tf.Session(graph=graph) as session:
    train_writer = tf.summary.FileWriter(os.path.join(logs_path, 'train'), graph=session.graph)
    valid_writer = tf.summary.FileWriter(os.path.join(logs_path, 'valid'))
    
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps):
        
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict={
            tf_dataset : batch_data, 
            tf_labels : batch_labels
        }
        
        _, l, prediction = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 5000 == 0):
            summary, train_acc, valid_acc = session.run([merged_summary, train_accuracy, valid_accuracy],
                                                        feed_dict=feed_dict)
            train_writer.add_summary(summary, step)
            valid_writer.add_summary(summary, step)
            
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.2f%%' % train_acc)
            print('Validation accuracy: %.2f%%' % valid_acc)                
                            
    test_accuracy = accuracy(test_prediction.eval(), test_labels) 
    print('Test accuracy: %.2f%%' % test_accuracy)


Initialized
Minibatch loss at step 0: 2.309797
Minibatch accuracy: 0.00%
Validation accuracy: 12.59%
Minibatch loss at step 5000: 1.274386
Minibatch accuracy: 68.75%
Validation accuracy: 78.34%
Minibatch loss at step 10000: 0.601179
Minibatch accuracy: 68.75%
Validation accuracy: 80.78%
Minibatch loss at step 15000: 0.854389
Minibatch accuracy: 81.25%
Validation accuracy: 82.04%
Minibatch loss at step 20000: 0.814152
Minibatch accuracy: 68.75%
Validation accuracy: 83.20%
Minibatch loss at step 25000: 0.632932
Minibatch accuracy: 75.00%
Validation accuracy: 83.68%
Minibatch loss at step 30000: 0.638202
Minibatch accuracy: 81.25%
Validation accuracy: 84.28%
Minibatch loss at step 35000: 0.350202
Minibatch accuracy: 81.25%
Validation accuracy: 84.49%
Test accuracy: 91.35%
